In [27]:
# If you do not have plotly installed, uncomment the line and run this cell
# Install kaleido for writing the maps to a png file

# !pip install plotly
# !pip install -U kaleido

# To retrieve the topojson github, uncomment the line and run the cell
# !git clone https://github.com/sgillies/topojson.git

In [28]:
import os
import numpy as np
import pandas as pd
import topojson  
from topojson import topojson
import json
import requests
import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.io as pio
from kaleido.scopes.plotly import PlotlyScope

#### Much of the code here was adapted for our project from this Texas bivariate choropleth map example:
#### https://chart-studio.plotly.com/~empet/15191/texas-bivariate-choropleth-assoc/#/

#### NOTE: While the topojson map of Europe includes all european countries, our covid data only cover European Union countries


In [29]:
'''
Function assigns what each countries color will be for each week

Parameters:
delta_data, var_data: lists or 1d arrays, containing values of the two variables
*_bound_* variables: bounds that determine what data values will be mapped to which color 
biv_colors: contains color code


'''

def data_to_color(delta_data, var_data, delta_bound_1, delta_bound_2, var_bound_1, var_bound_2, biv_colors):
    # This function works only with a list of 9 bivariate colors
    
    if len(delta_data) != len(var_data):
        raise ValueError('the list of delta_data and var_data must have the same length')
    n_colors = len(biv_colors)
    
    if n_colors != 9:
        raise ValueError('the list of bivariate colors must have the length equal to 9')
    
    num = 3
    xcol = [set_interval_value(vals, var_bound_1, var_bound_2) for vals in var_data]
    ycol = [set_interval_value(vals, delta_bound_1, delta_bound_2) for vals in delta_data]
    idxcol = [int(xcolor + num*ycolor) for xcolor, ycolor in zip(xcol,ycol)]# index of the corresponding color in the list of bivariate colors
    colors = np.array(biv_colors)[idxcol]
    
    return list(colors)

In [30]:
# Function assigns the countries float value to an integer bound

def set_interval_value(vals, bound_1, bound_2):
    if vals <= bound_1:
        return 0
    elif bound_1 < vals <= bound_2:
        return 1
    else:
        return 2

In [31]:
# Makes the colorsquare legend for the bivariate map

def colorsquare(delta_text, var_text, colorscale, num=3, xaxis='x2', yaxis='y2'):

    z = [[j + num * i for j in range(num)] for i in range(num)]
    n = len(delta_text)
    if len(delta_text) != n or len(var_text) != n or len(colorscale) != 2 * n ** 2:
        raise ValueError('Your lists of strings  must have the length {n} and the colorscale, {n**2}')

    text = [[delta_text[j] + '<br>' + var_text[i] for j in range(len(delta_text))] for i in range(len(var_text))]
    return go.Heatmap(x=list(range(n)),
                      y=list(range(n)),
                      z=z,
                      xaxis=xaxis,
                      yaxis=yaxis,
                      text=text,
                      hoverinfo='text',
                      colorscale=colorscale,
                      showscale=False)

In [32]:
# Returns a discrete colorscale defined by biv_colors

def colors_to_colorscale(biv_colors):
    num = len(biv_colors)
    biv_colorscale = []
    
    for i, col in enumerate(biv_colors):
        biv_colorscale.extend([[round(i/num, 2) , col], [round((i+1)/num, 2), col]])
    
    return biv_colorscale

In [61]:
# Creates the map for each week given the dataframe, the color scheme, the variant to show with 
# delta, the country, week, and bounds

def generate_map(prep_map_data, bi_color, var_type, all_countries, week, bounds):

    # extract the values for the delta column and the other variants column
    delta_list = [prep_map_data.loc[country, 'Delta'] for country in all_countries]
    var_list = [prep_map_data.loc[country, var_type] for country in all_countries]

    # assigns each data point in the lists to the appropriate bivariate color
    country_color = data_to_color(delta_list, var_list, bounds[0], bounds[1], bounds[0],
                                  bounds[1], bi_color)

    # text for interactive map when cursor is hovered over a country
    text = [c + '<br>Delta Cases Percent: ' + '{:0.4f}'.format(d) + '<br>' + var_type + ' Cases Percent: ' + '{:0.4f}'.format(v)
            for c, d, v in zip(countries, delta_list, var_list)]

    # creating each country object
    country_centers = dict(type='scatter',
                          y=lats,
                          x=lons,
                          mode='markers',
                          text=text,
                          marker=dict(size=1, color=country_color),
                          showlegend=False,
                          hoverinfo='text')

    # defining the map geometrical boundaries for the countries and filling it with color
    n = len(bi_color)
    data = []
    fc = np.array(country_color)
    for num in range(9):
        idx_color = np.where(fc == bi_color[num])[0]
        for i in idx_color:
            pts = []
            feature = geoJSON['features'][i]
            if feature['geometry']['type'] == 'Polygon':
                pts.extend(feature['geometry']['coordinates'][0])
                pts.append([None, None])  # mark the end of a polygon

            elif feature['geometry']['type'] == 'MultiPolygon':
                for polyg in feature['geometry']['coordinates']:
                    pts.extend(polyg[0])
                    pts.append([None, None])  # end of polygon
            else:
                raise ValueError("geometry type irrelevant for a map")

            coor_x, coor_y = zip(*pts)
            data.append(dict(type='scatter',
                             x=coor_x, y=coor_y,
                             fill='toself',
                             fillcolor=bi_color[num],  # country_color[i],
                             hoverinfo='none',
                             mode='lines',
                             line=dict(width=1, color='rgb(150,150,150)'),
                             opacity=0.95)
                        )
    data.append(country_centers)

    # text for interactive bivariate legend
    delta_text = ['Delta < P_33', 'P_33 <= Delta <= P_66 ', 'Delta > P_66']
    var_text = [var_type + ' < P_33', 'P_33 <= ' + var_type + ' <= P_66 ', var_type + ' > P_66']

    #creating the legend
    legend = colorsquare(var_text, delta_text, colors_to_colorscale(bi_color))
    data.append(legend)

    legend_axis = dict(showline=False, zeroline=False, showgrid=False, ticks='', showticklabels=False)

    layout = dict(title=week,
                  font=dict(family='Balto'),
                  titlefont=dict(size=32),
                  title_x=0.5,
                  showlegend=False,
                  hovermode='closest',
                  xaxis=dict(autorange=False,
                             range=[-29.1, 55],  # an interval of lons that covers Europe
                             domain=[0, 1],
                             showgrid=False,
                             zeroline=False,
                             fixedrange=True,
                             ticks='',
                             showticklabels=False),
                  yaxis=dict(autorange=False,
                             range=[34, 72.1],  # an interval of lats that cover Europe
                             domain=[0, 1],
                             showgrid=False,
                             zeroline=False,
                             ticks='',
                             showticklabels=False,
                             fixedrange=True),
                  xaxis2=dict(legend_axis, **dict(domain=[0.065, 0.25],
                                                  anchor='free',
                                                  side='bottom',
                                                  position=0.65,
                                                  title='    ' + str(round(bounds[0], 3)) + '      ' + str(round(bounds[1], 3)) + '<br>' + var_type + ' Cases Per 100,000',
                                                  titlefont=dict(size=16))),
                  yaxis2=dict(legend_axis, **dict(domain=[.61, 0.75],
                                                  anchor='free',
                                                  side='left',
                                                  position=0.075,
                                                  title='Delta Cases Per 100,000<br> ' + str(round(bounds[0], 3)) + '    ' + str(round(bounds[1], 3)),
                                                  titlefont=dict(size=16))),
                  width=1100,
                  height=850,
                  dragmode='select')

    # Taking the dict with all the data and layout settings and creating a png
    fig = go.Figure(dict(data=data, layout = layout))
    
    newpath = 'png_files' 
    if not os.path.exists(newpath):
        os.makedirs(newpath)

    with open('png_files/' + week + '.png', 'w'):
        pio.write_image(fig=fig, file='png_files/' + week + '.png', format='png')

    #pyo.plot(fig, filename=week +'.html') #THIS FOR INTERACTIVE HTML figures

### Load the JSON map of Europe

In [34]:
url_europe = 'https://gist.githubusercontent.com/milafrerichs/69035da4707ea51886eb/raw/4cb1783c2904f52cbb8a258ee96031f9054d155b/eu.topojson'
my_file = requests.get(url_europe)
topoJSON = json.loads(my_file.content)

topo_features = topoJSON['objects']['europe']['geometries']
scale = topoJSON['transform']['scale']
translation = topoJSON['transform']['translate']

### Convert Europe topoJSON data into a geojson dictionary

In [35]:
geoJSON = dict(type= 'FeatureCollection',
             features = [])

for k, tfeature in enumerate(topo_features):
    geo_feature = dict(id=k, type= "Feature")
    geo_feature['properties'] = tfeature['properties']
    geo_feature['id'] = tfeature['properties']['iso_a3']
    geo_feature['geometry'] = topojson.geometry(tfeature, topoJSON['arcs'], scale, translation)
    geoJSON['features'].append(geo_feature)

### Get the longitude, latitude, and country name for each country

In [36]:
lons = []
lats = []
for k in range(len(geoJSON['features'])):
    country_coords = np.array(geoJSON['features'][k]['geometry']['coordinates'][0])
    m, M = country_coords[:,0].min(), country_coords[:,0].max()
    lons.append(0.5*(m+M))
    m, M = country_coords[:,1].min(), country_coords[:,1].max()
    lats.append(0.5*(m+M))
    
# Code to get country name and id lists and the total number of countries
countries = [geoJSON['features'][k]['properties']['name'] for k in range(len(geoJSON['features']))]
countries_df = pd.DataFrame (countries, columns = ['Country'])
country_ids = [geoJSON['features'][k]['id'] for k in range(len(geoJSON['features']))]
num_countries = len(geoJSON['features'])

### Create Bivariate Choropleth Maps for each week

In [37]:
# Bivariate color codes: Blue = delta, Red = Omicron, Yellow = alpha
alp_del = ["#efefeb", "#dede97", "#caca3b", "#8597ef", "#7b8c99", "#70803c", "#002af5", "#00279d", "#00233d"]
del_omi = ["#efefeb", "#f0a390", "#f2340e", "#8597ef", "#856792", "#86210f", "#002af5", "#001c96", "#00090f"]

In [60]:
# Code to make maps by week
# Feeds data into map function after determining if alpha or delta is most prominent and creating comprehensive df
url_data = 'https://raw.githubusercontent.com/bryn-m/Covid_Evolution/main/tidy_variant_data.tsv'
variant_data = pd.read_csv(url_data, sep='\t')

variant_df = pd.DataFrame(variant_data)

# creating the bounds for assigning the bivariate colors
bound_list = np.array(variant_df[['Delta', 'Alpha', 'Omicron']].values.tolist())
bound_list = bound_list.flatten()
bound_list = bound_list[bound_list != 0.0]

bounds = np.percentile(bound_list, [33, 66])

# grouping the data by week
by_week_dfs = variant_data.groupby(by='Year_Week')
weeks = variant_data['Year_Week'].unique()

# determines the variant to display on the map with the delta variant based on the greatest average cases
# creates a dataframe with the country name, delta variant, and the second determined variant to then feed into the map creating function
# this is done for all weeks
for week in weeks:
    cur_week_df = by_week_dfs.get_group(week)

    delta = cur_week_df.loc[:,['Country', 'Delta']]
    temp_alpha = cur_week_df.loc[:, 'Alpha']
    temp_omicron = cur_week_df.loc[:, 'Omicron']
    if temp_alpha.mean() > temp_omicron.mean():
        alpha = cur_week_df.loc[:,['Country', 'Alpha']]
        complete_df = pd.merge(countries_df, alpha, on='Country', how='outer')
        color = alp_del
        var = 'Alpha'
    else:
        omicron = cur_week_df.loc[:, ['Country', 'Omicron']]
        complete_df = pd.merge(countries_df, omicron, on='Country', how='outer')
        color = del_omi
        var = 'Omicron'

    complete_df = pd.merge(complete_df, delta, on='Country', how='outer')
    complete_df.fillna(0, inplace=True)
    complete_df.set_index('Country', drop=True, inplace=True)

    generate_map(complete_df, color, var, countries, week, bounds)